In [55]:
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import yfinance as yf
from scipy import stats


# 1. Load data
# factors.csv expected columns: Date, MKT, SMB, HML, RF  (Date as YYYY-MM or YYYY-MM-DD)
# asset_returns.csv expected columns: Date, AssetID, Return  (Return in decimal, e.g., 0.02 for 2%)

In [37]:
# 1. Specify tickers and date range
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA']  # example; extend as needed
start_date = '2010-01-01'
end_date = '2024-12-31'

# 2. Download daily adjusted close prices from Yahoo
#    yfinance returns daily data by default
data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']

/var/folders/hx/ckgg6j1s22l73jpkp3g7np640000gn/T/ipykernel_45726/2719945950.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']


In [38]:
#    First, resample to month-end prices
monthly_prices = data.resample('M').last()
# Compute simple returns: pct change month to month
monthly_returns = monthly_prices.pct_change().dropna(how='all')
# Convert to long format for merging later
monthly_ret_long = monthly_returns.reset_index().melt(id_vars='Date', var_name='AssetID', value_name='Return')
# Drop any NaNs
monthly_ret_long = monthly_ret_long.dropna(subset=['Return'])

monthly_ret_long


/var/folders/hx/ckgg6j1s22l73jpkp3g7np640000gn/T/ipykernel_45726/903371793.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_prices = data.resample('M').last()


,Date,AssetID,Return
0,2010-02-28,AAPL,0.065396
1,2010-03-31,AAPL,0.148471
2,2010-04-30,AAPL,0.111021
3,2010-05-31,AAPL,-0.016125
4,2010-06-30,AAPL,-0.020827
...,...,...,...
890,2024-08-31,TSLA,-0.077390
891,2024-09-30,TSLA,0.221942
892,2024-10-31,TSLA,-0.045025
893,2024-11-30,TSLA,0.381469


In [39]:
ff = pd.read_csv('F-F_Research_Data_Factors.csv', skiprows=3)  # skip header rows as per file format
# Often the CSV has a footer; drop rows where Date is NaN or non-numeric
ff = ff.rename(columns=lambda x: x.strip())
# Keep until a row where Date is not in YYYYMM format. For example:
ff = ff[ff['Date'].astype(str).str.match(r'^\d{6}$')]
# Parse Date: if format is YYYYMM, convert to datetime at month-end
ff['YearMonth'] = pd.to_datetime(ff['Date'].astype(str), format='%Y%m') + pd.offsets.MonthEnd(0)
# Convert numeric columns to floats and scale: French data usually in percentages, e.g., 0.5 means 0.5%
for col in ['Mkt-RF', 'SMB', 'HML', 'RF']:
    ff[col] = pd.to_numeric(ff[col], errors='coerce') / 100.0
# Compute Excess Market if needed: often Mkt-RF is already excess market return
# So ExcessMKT = Mkt-RF; if not, do: ff['ExcessMKT'] = ff['Mkt'] - ff['RF']
ff['ExcessMKT'] = ff['Mkt-RF']
# Keep relevant columns
ff = ff[['YearMonth', 'ExcessMKT', 'SMB', 'HML', 'RF']]


In [40]:
# 5. Prepare asset returns with excess returns
# Align monthly_ret_long Date to month-end Period
monthly_ret_long['YearMonth'] = monthly_ret_long['Date'].dt.to_period('M').dt.to_timestamp('M')
# Merge with RF to get excess returns
asset_data = monthly_ret_long.merge(ff[['YearMonth', 'RF']], on='YearMonth', how='left')
asset_data['ExcessReturn'] = asset_data['Return'] - asset_data['RF']
# Drop rows without factor data
asset_data = asset_data.dropna(subset=['ExcessReturn'])

In [41]:
asset_data

,Date,AssetID,Return,YearMonth,RF,ExcessReturn
0,2010-02-28,AAPL,0.065396,2010-02-28,0.0000,0.065396
1,2010-03-31,AAPL,0.148471,2010-03-31,0.0001,0.148371
2,2010-04-30,AAPL,0.111021,2010-04-30,0.0001,0.110921
3,2010-05-31,AAPL,-0.016125,2010-05-31,0.0001,-0.016225
4,2010-06-30,AAPL,-0.020827,2010-06-30,0.0001,-0.020927
...,...,...,...,...,...,...
885,2024-08-31,TSLA,-0.077390,2024-08-31,0.0048,-0.082190
886,2024-09-30,TSLA,0.221942,2024-09-30,0.0040,0.217942
887,2024-10-31,TSLA,-0.045025,2024-10-31,0.0039,-0.048925
888,2024-11-30,TSLA,0.381469,2024-11-30,0.0040,0.377469


In [42]:
# Pivot returns: index = YearMonth, columns = AssetID
ret_wide = asset_data.pivot(index='YearMonth', columns='AssetID', values='ExcessReturn')
# Factor series aligned: index = YearMonth
fac = ff.set_index('YearMonth')[['ExcessMKT', 'SMB', 'HML']]

In [43]:
display(fac)
display(ret_wide)

,ExcessMKT,SMB,HML
YearMonth,,,
1926-07-31,0.0289,-0.0255,-0.0239
1926-08-31,0.0264,-0.0114,0.0381
1926-09-30,0.0038,-0.0136,0.0005
1926-10-31,-0.0327,-0.0014,0.0082
1926-11-30,0.0254,-0.0011,-0.0061
...,...,...,...
2024-12-31,-0.0317,-0.0272,-0.0300
2025-01-31,0.0280,-0.0195,0.0162
2025-02-28,-0.0244,-0.0581,0.0491


AssetID,AAPL,AMZN,GOOG,MSFT,TSLA
YearMonth,,,,,
2010-02-28,0.065396,-0.055897,-0.005925,0.022145,NaN
2010-03-31,0.148371,0.146606,0.076438,0.021526,NaN
2010-04-30,0.110921,0.009696,-0.073136,0.042577,NaN
2010-05-31,-0.016225,-0.085002,-0.076322,-0.151494,NaN
2010-06-30,-0.020927,-0.129225,-0.083867,-0.108239,NaN
...,...,...,...,...,...
2024-08-31,0.027554,-0.050152,-0.051234,-0.005895,-0.082190
2024-09-30,0.013467,0.039866,0.009931,0.027548,0.217942
2024-10-31,-0.034329,-0.003524,0.028997,-0.059559,-0.048925


In [45]:
window = 60
beta_records = []
all_months = sorted(ret_wide.index)
for i, current_month in enumerate(all_months):
    if i < window:
        continue
    # Use past `window` months (i-window to i-1)
    hist_months = all_months[i-window : i]
    X_hist = fac.loc[hist_months]
    if X_hist.isna().any().any():
        # If factor data missing, skip this month
        continue
    X_mat = sm.add_constant(X_hist.values)  # shape (window, 4)
    for asset in ret_wide.columns:
        y_hist = ret_wide.loc[hist_months, asset]
        if y_hist.isna().any():
            continue  # skip assets with missing returns in window
        # OLS time-series regression
        model = sm.OLS(y_hist.values, X_mat)
        res = model.fit()
        params = res.params  # [const, beta_mkt, beta_smb, beta_hml]
        beta_records.append({
            'YearMonth': current_month,
            'AssetID': asset,
            'beta_const': params[0],
            'beta_mkt': params[1],
            'beta_smb': params[2],
            'beta_hml': params[3]
        })

beta_df = pd.DataFrame(beta_records)
beta_df.set_index(['YearMonth','AssetID'], inplace=True)

In [46]:
beta_df

beta_const  beta_mkt  beta_smb  beta_hml
YearMonth  AssetID                                          
2015-02-28 AAPL       0.014177  1.040374 -0.576861 -0.681619
           AMZN       0.004247  1.122165  0.240139 -1.324983
           GOOG      -0.001366  1.223952 -0.678157 -0.351638
           MSFT      -0.002507  1.045526 -0.645272  0.337918
2015-03-31 AAPL       0.013300  1.056625 -0.600965 -0.769801
...                        ...       ...       ...       ...
2024-12-31 AAPL       0.009294  1.218981 -0.117516 -0.488545
           AMZN       0.003381  1.158828  0.041101 -0.940327
           GOOG       0.005084  1.037079 -0.284659 -0.249240
           MSFT       0.006996  0.952177 -0.529616 -0.511544
           TSLA       0.042762  2.189096  1.734129 -1.504674

[590 rows x 4 columns]

In [50]:
# 7. Cross-sectional regression period by period with OLS and LPQ adjustment
results = []
for current_month in sorted(set(beta_df.index.get_level_values('YearMonth'))):
    b = beta_df.loc[current_month]
    # Gather realized excess returns in current_month
    rets = ret_wide.loc[current_month]
    df_cs = b.join(rets.rename('ExRet'), how='inner').dropna(subset=['ExRet'])
    n = df_cs.shape[0]
    p = 3  # number of factors
    if n < (p+1):
        continue
    # Build X and y
    X = np.column_stack([
        np.ones(n),
        df_cs['beta_mkt'].values,
        df_cs['beta_smb'].values,
        df_cs['beta_hml'].values
    ])
    y = df_cs['ExRet'].values
    # OLS
    XtX = X.T @ X
    XtX_inv = np.linalg.pinv(XtX)
    X_pinv = XtX_inv @ X.T
    gamma_ols = X_pinv @ y  # length 4
    # Residuals
    z = y - X @ gamma_ols
    M = np.eye(n) - X @ X_pinv
    # Moments
    if n <= (p + 1):
        # not enough degrees of freedom to estimate residual variance
        continue
    sigma2_hat = np.sum(z**2) / (n - (p+1))
    z_centered = z - np.mean(z)
    s2 = np.mean(z_centered**2)
    if s2 <= 1e-8:
        continue
    m3_hat = np.mean(z_centered**3) / (s2**1.5)
    m4_hat = np.mean(z_centered**4) / (s2**2) - 3
    # LPQ adjustment
    M_star = M * M
    inv_M_star = np.linalg.pinv(M_star)
    K_mat = 2 * inv_M_star + m4_hat*np.eye(n) - (m3_hat**2)*M
    inv_K = np.linalg.pinv(K_mat)
    term = m3_hat*z - (1/np.sqrt(sigma2_hat))*(inv_M_star @ (z*z)) + np.sqrt(sigma2_hat)*np.ones(n)
    adjustment = m3_hat * (X_pinv @ (inv_K @ term))
    gamma_lpq = gamma_ols + adjustment
    # Store
    rec = {
        'YearMonth': current_month,
        'n_assets': n,
        'gamma0_ols': gamma_ols[0],
        'gamma_mkt_ols': gamma_ols[1],
        'gamma_smb_ols': gamma_ols[2],
        'gamma_hml_ols': gamma_ols[3],
        'gamma0_lpq': gamma_lpq[0],
        'gamma_mkt_lpq': gamma_lpq[1],
        'gamma_smb_lpq': gamma_lpq[2],
        'gamma_hml_lpq': gamma_lpq[3],
    }
    results.append(rec)

results_df = pd.DataFrame(results)
results_df['YearMonth'] = pd.to_datetime(results_df['YearMonth'].astype(str))
results_df.set_index('YearMonth', inplace=True)

In [51]:
results_df

,n_assets,gamma0_ols,gamma_mkt_ols,gamma_smb_ols,gamma_hml_ols,gamma0_lpq,gamma_mkt_lpq,gamma_smb_lpq,gamma_hml_lpq
YearMonth,,,,,,,,,
2015-07-31,5,-1.200800,1.447444,0.368497,0.040940,-1.226950,1.472363,0.370858,0.039966
2015-08-31,5,-0.340011,0.314458,0.061508,0.018067,-0.349044,0.323479,0.063995,0.019404
2015-09-30,5,0.226116,-0.253694,-0.040399,-0.009718,0.215817,-0.242216,-0.038282,-0.008392
2015-10-31,5,2.823578,-2.931764,-0.333802,0.038897,1.654391,-1.712711,-0.248638,0.013994
2015-11-30,5,-0.446293,0.570401,0.113439,0.027807,-0.272528,0.370269,0.073978,0.041946
...,...,...,...,...,...,...,...,...,...
2024-08-31,5,-0.760206,0.568357,-0.380205,-0.026567,-0.766510,0.573744,-0.383556,-0.026509
2024-09-30,5,-0.138027,0.120685,0.003492,-0.058258,-0.137983,0.120650,0.003517,-0.058252
2024-10-31,5,0.747316,-0.571443,0.370695,0.114837,0.747199,-0.571346,0.370631,0.114826


In [52]:
# 8. Aggregate over time
summary = {}
for prefix in ['gamma0','gamma_mkt','gamma_smb','gamma_hml']:
    ols_series = results_df[f'{prefix}_ols']
    lpq_series = results_df[f'{prefix}_lpq']
    T = len(ols_series)
    if T == 0:
        continue
    avg_ols = ols_series.mean()
    avg_lpq = lpq_series.mean()
    se_ols = ols_series.std(ddof=1) / np.sqrt(T)
    se_lpq = lpq_series.std(ddof=1) / np.sqrt(T)
    summary[prefix] = {
        'avg_ols': avg_ols, 'se_ols': se_ols,
        'avg_lpq': avg_lpq, 'se_lpq': se_lpq
    }
summary_df = pd.DataFrame(summary).T
print("Fama–MacBeth average premia (OLS vs LPQ):")
print(summary_df)

Fama–MacBeth average premia (OLS vs LPQ):
            avg_ols    se_ols   avg_lpq    se_lpq
gamma0     0.024202  0.042619 -0.007170  0.039728
gamma_mkt  0.013315  0.042003  0.037952  0.037086
gamma_smb  0.012788  0.037688  0.005673  0.039708
gamma_hml  0.022026  0.018271  0.022345  0.018368


In [56]:
# Assume you have for month t: a DataFrame df_cs with columns ['ExRet','beta_mkt','beta_smb','beta_hml']
# and you have gamma arrays gamma_ols, gamma_lpq for that month.
# You can loop over months to compute RMSE series.

rmse_records = []
for current_month in results_df.index:
    # Retrieve betas and returns for that month:
    # Suppose beta_df and ret_wide exist as before
    try:
        b = beta_df.loc[current_month]
        rets = ret_wide.loc[current_month]
    except KeyError:
        continue
    df_cs = b.join(rets.rename('ExRet'), how='inner').dropna(subset=['ExRet'])
    n = df_cs.shape[0]
    if n <= 0:
        continue
    # Get gamma estimates
    gamma_ols = results_df.loc[current_month, ['gamma0_ols','gamma_mkt_ols','gamma_smb_ols','gamma_hml_ols']].values
    gamma_lpq = results_df.loc[current_month, ['gamma0_lpq','gamma_mkt_lpq','gamma_smb_lpq','gamma_hml_lpq']].values
    # Predicted returns
    X = np.column_stack([
        np.ones(n),
        df_cs['beta_mkt'].values,
        df_cs['beta_smb'].values,
        df_cs['beta_hml'].values
    ])
    pred_ols = X @ gamma_ols
    pred_lpq = X @ gamma_lpq
    actual = df_cs['ExRet'].values
    e_ols = actual - pred_ols
    e_lpq = actual - pred_lpq
    rmse_ols = np.sqrt(np.mean(e_ols**2))
    rmse_lpq = np.sqrt(np.mean(e_lpq**2))
    # R-squared
    ss_tot = np.sum((actual - actual.mean())**2)
    r2_ols = 1 - np.sum(e_ols**2)/ss_tot if ss_tot>0 else np.nan
    r2_lpq = 1 - np.sum(e_lpq**2)/ss_tot if ss_tot>0 else np.nan
    rmse_records.append({
        'YearMonth': current_month,
        'rmse_ols': rmse_ols,
        'rmse_lpq': rmse_lpq,
        'r2_ols': r2_ols,
        'r2_lpq': r2_lpq
    })

rmse_df = pd.DataFrame(rmse_records).set_index('YearMonth')
# Compare average RMSE and R2
avg_rmse = rmse_df[['rmse_ols','rmse_lpq']].mean()
avg_r2 = rmse_df[['r2_ols','r2_lpq']].mean()
print("Average RMSE:", avg_rmse)
print("Average R2:", avg_r2)

# Paired t-test on RMSE differences
diff_rmse = rmse_df['rmse_lpq'] - rmse_df['rmse_ols']
T2 = len(diff_rmse.dropna())
mean_diff = diff_rmse.mean(); se_diff = diff_rmse.std(ddof=1)/np.sqrt(T2)
t_rmse = mean_diff / se_diff
p_rmse = 2*(1 - stats.t.cdf(abs(t_rmse), df=T2-1))
print(f"RMSE difference mean: {mean_diff:.4f}, t={t_rmse:.2f}, p={p_rmse:.3f}")

Average RMSE: rmse_ols    0.019881
rmse_lpq    0.024120
dtype: float64
Average R2: r2_ols    0.825016
r2_lpq    0.555474
dtype: float64
RMSE difference mean: 0.0042, t=2.56, p=0.012
